In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

Reading dataset

In [ ]:
dataset = pd.read_csv("../input/students-performance-in-exams/StudentsPerformance.csv")
X = dataset.drop(['math score', 'lunch','race/ethnicity'],1)
y = dataset['math score'].values
y = y.reshape(len(y),1)


# Data preprocessing

Handling catogrical variables using colomun transformer, one hot encoding

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
ct = ColumnTransformer(transformers = [("encoder1", OneHotEncoder(),['parental level of education']), ("encoder2", OneHotEncoder(),['gender']),("encoder3", OneHotEncoder(),['test preparation course']), ("encoder4", OneHotEncoder(),['test preparation course'])], remainder ="passthrough")
X = np.array(ct.fit_transform(X))

ct.transformers_

spliting the dataset using train_test_split

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 5)


scaling the dataset using Standard scaller

In [ ]:
from sklearn.preprocessing import StandardScaler
scx = StandardScaler()
scy = StandardScaler()
X_train[:,:12]=scx.fit_transform(X_train[:,:12])
X_test[:,:12]= scx.transform(X_test[:,:12])
y_train = scy.fit_transform(y_train)

# Model building

using svr for regression

In [ ]:
from sklearn.svm import SVR
regressor = SVR(kernel = 'rbf')
regressor.fit(X_train, y_train.ravel())

# Making prediction

In [ ]:
y_pred = scy.inverse_transform(regressor.predict(X_test))
np.set_printoptions(precision = 2)
print(y_pred)
print(np.concatenate((y_pred.reshape(len(y_pred),1),y_test.reshape(len(y_test),1)),1))

Preformance measurement using r2socre

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

cross validation of model using srocc_val_score

In [ ]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = regressor, X = X_test, y= y_test.ravel(), cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard davition: {:.2f}%".format(accuracies.std()*100))


# Hyperparameter tuning
 Using Grid_search_cv

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters = [{'C': [0.25, 0.5, 0.75, 1], 'kernel': ['linear']},
              {'C': [0.25, 0.5, 0.75, 1], 'kernel': ['rbf'], 'gamma': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8,0.9,]}]
grid_search = GridSearchCV(estimator = regressor,
                          param_grid = parameters,
                          scoring = 'r2',
                           n_jobs = -1)
grid_search.fit(X_train, y_train.ravel())
best_accuracy = grid_search.best_score_
best_prameter = grid_search.best_params_
print("best accuracy: {:.2f}%". format(best_accuracy*100))
print("best parameter ", best_prameter)